<a href="https://colab.research.google.com/github/paramate-p/selected-topic-cmu/blob/main/Lecture02%20/workshop2/workshop_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install textstat

In [6]:
!pip install datasets

In [7]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textstat import flesch_kincaid_grade
from datasets import load_dataset

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [26]:
# Load the IMDB dataset
dataset = load_dataset("imdb")

# combine data
train_data = pd.DataFrame(dataset['train'])
test_data = pd.DataFrame(dataset['test'])
data = pd.concat([train_data, test_data], ignore_index=True)

data.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [15]:
def clean_text(text):
    text = re.sub(r'[\U00010000-\U0010FFFF]', '', text) # remove emoji that matches any character in the unicode range from U+10000 to U+10FFFF
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Replace special characters and numbers with a space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

data['cleaned_review'] = data['text'].apply(clean_text)

# Tokenization
data['sentences'] = data['cleaned_review'].apply(sent_tokenize)
data['words'] = data['cleaned_review'].apply(word_tokenize)

# Lowercasing and stop word removal
stop_words = set(stopwords.words('english'))

def remove_stopwords(word_list):
    return [word.lower() for word in word_list if word.lower() not in stop_words]

data['filtered_words'] = data['words'].apply(remove_stopwords)

# Stemming & lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stemming(words):
    return [stemmer.stem(word) for word in words]

def lemmatization(words):
    return [lemmatizer.lemmatize(word) for word in words]

data['stemmed_words'] = data['filtered_words'].apply(stemming)
data['lemmatized_words'] = data['filtered_words'].apply(lemmatization)

# Final: Calculate Flesch-Kincaid readability score
def calculate_readability(text):
    try:
        return flesch_kincaid_grade(text)
    except:
        return None

data['readability_score'] = data['cleaned_review'].apply(calculate_readability)

# Display a sample of the processed data
data[['text', 'cleaned_review', 'filtered_words', 'readability_score']].head()


Processing complete. Processed data saved as 'Processed_IMDB_Dataset.csv'.


In [18]:
data

,text,label,cleaned_review,sentences,words,filtered_words,stemmed_words,lemmatized_words,readability_score
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,I rented I AM CURIOUS YELLOW from my video sto...,[I rented I AM CURIOUS YELLOW from my video st...,"[I, rented, I, AM, CURIOUS, YELLOW, from, my, ...","[rented, curious, yellow, video, store, contro...","[rent, curiou, yellow, video, store, controver...","[rented, curious, yellow, video, store, contro...",116.0
1,"""I Am Curious: Yellow"" is a risible and preten...",0,I Am Curious Yellow is a risible and pretentio...,[I Am Curious Yellow is a risible and pretenti...,"[I, Am, Curious, Yellow, is, a, risible, and, ...","[curious, yellow, risible, pretentious, steami...","[curiou, yellow, risibl, pretenti, steam, pile...","[curious, yellow, risible, pretentious, steami...",90.6
2,If only to avoid making this type of film in t...,0,If only to avoid making this type of film in t...,[If only to avoid making this type of film in ...,"[If, only, to, avoid, making, this, type, of, ...","[avoid, making, type, film, future, film, inte...","[avoid, make, type, film, futur, film, interes...","[avoid, making, type, film, future, film, inte...",38.8
3,This film was probably inspired by Godard's Ma...,0,This film was probably inspired by Godard s Ma...,[This film was probably inspired by Godard s M...,"[This, film, was, probably, inspired, by, Goda...","[film, probably, inspired, godard, masculin, f...","[film, probabl, inspir, godard, masculin, f, m...","[film, probably, inspired, godard, masculin, f...",48.5
4,"Oh, brother...after hearing about this ridicul...",0,Oh brother after hearing about this ridiculous...,[Oh brother after hearing about this ridiculou...,"[Oh, brother, after, hearing, about, this, rid...","[oh, brother, hearing, ridiculous, film, umpte...","[oh, brother, hear, ridicul, film, umpteen, ye...","[oh, brother, hearing, ridiculous, film, umpte...",126.1
...,...,...,...,...,...,...,...,...,...
49995,Just got around to seeing Monster Man yesterda...,1,Just got around to seeing Monster Man yesterda...,[Just got around to seeing Monster Man yesterd...,"[Just, got, around, to, seeing, Monster, Man, ...","[got, around, seeing, monster, man, yesterday,...","[got, around, see, monster, man, yesterday, lo...","[got, around, seeing, monster, man, yesterday,...",300.8
49996,I got this as part of a competition prize. I w...,1,I got this as part of a competition prize I wa...,[I got this as part of a competition prize I w...,"[I, got, this, as, part, of, a, competition, p...","[got, part, competition, prize, watched, reall...","[got, part, competit, prize, watch, realli, ex...","[got, part, competition, prize, watched, reall...",49.3
49997,I got Monster Man in a box set of three films ...,1,I got Monster Man in a box set of three films ...,[I got Monster Man in a box set of three films...,"[I, got, Monster, Man, in, a, box, set, of, th...","[got, monster, man, box, set, three, films, ma...","[got, monster, man, box, set, three, film, mai...","[got, monster, man, box, set, three, film, mai...",83.2
49998,"Five minutes in, i started to feel how naff th...",1,Five minutes in i started to feel how naff thi...,[Five minutes in i started to feel how naff th...,"[Five, minutes, in, i, started, to, feel, how,...","[five, minutes, started, feel, naff, looking, ...","[five, minut, start, feel, naff, look, got, co...","[five, minute, started, feel, naff, looking, g...",150.7
